# TESTING ONE EVENT: Microsoft MA announcement Yahoo_data

## 0. Libraries, functions and model specification

In [2]:
#Libraries
import pandas as pd
import numpy as np
import pickle
from pandas.tseries.offsets import *   
from itertools import chain
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from scipy import stats

In [3]:
# %load buildD.py
def buildD(pair):
    #1.window parametrization
    #estimation window (T1:train-150 and T2:test-50: 200); T3:gap-25; event-21(T4:-10:0:10T5)
    #[T1,T2): train; [T2,T3):test; [T3,T4):gap; [T4,T5]:event
    eventw=10 #size of the event window
    ztrain=150
    ztest=50
    zgap=25
    zevent=2*eventw+1
    # 2. get stock involved in event and mkt data (traditional)
    assethis=st16f.loc[(slice(None),pair['permno'])]
    evindex=assethis.index.get_loc((pair['date']))
    #using event index get dates for event study windows
    T1=assethis.index[evindex-(ztrain+ztest+zgap+eventw)]
    T2=assethis.index[evindex-(ztest+zgap+eventw)]
    T3=assethis.index[evindex-(zgap+eventw)]
    T4=assethis.index[evindex-(eventw)]
    T5=assethis.index[evindex+(eventw)]
    dinvl=(T1,T2,T3,T4,pair['date'],T5)
    #sample data for stock of interest and market for window
    assetsmp=assethis.loc[(slice(dinvl[0],dinvl[len(dinvl)-1]))]
    sprtsmp=spmktix.loc[(slice(dinvl[0],dinvl[len(dinvl)-1]))]
    assetsmp=pd.DataFrame(np.matrix((assetsmp.values,sprtsmp.values)).reshape(2,len(assetsmp)).transpose(),
                      index=assetsmp.index,columns=(pair['permno'],'sprtrn'))
    #clean data
    del assethis 
    del sprtsmp
    #3.get control group for stocks using sample selection restrictions (synthetic matching)
    #Select permno with event in estimation window
    permnoex=EmadteOKS.loc[(slice(dinvl[0],dinvl[2]))].unique()
    #append permno of the asset of interest
    permnoex=np.append(permnoex,pair['permno'])
    permnoex.sort(axis=-1)
    permnoex=pd.Series(permnoex)  #check date event in tail
    #Select data for all assets using the relevent dates for event
    # Estimation + Event window absolute slicing
    allafw=st16f.loc[(slice(dinvl[0],dinvl[len(dinvl)-1]),slice(None))]
    # Select only assets that are not involved in event during estimation window
    #Get permno for all assets availables in the data
    permnoad=allafw.index.droplevel(level='date').unique()
    permnoad=pd.Series(permnoad)
    #select permno that are not affected by event
    permnoacf=permnoad[~permnoad.isin(permnoex)]
    #Keep only assets for the control group that are not involed in similar events
    allafwn=allafw.reset_index(level=['date'])
    allafwn.columns=['date','RETX']
    #create logistic to select permno's not affected by event
    conafw=allafwn.iloc[allafwn.index.isin(permnoacf)]
    #Change structure of the data using pivoting
    conafw.reset_index(inplace=True)
    #pivot the table
    conafwp=conafw.pivot(index='date',columns='PERMNO',values='RETX')
    #Remove assets with NaN values
    conafwp=conafwp.loc[:,conafwp.notnull().all()]
    #4. merge data
    #concatenate columns to create estimation window 
    Fwdow=pd.concat([assetsmp,conafwp],axis=1)
    outdt=(dinvl,Fwdow)
    return outdt

In [4]:
# %load rsynth.py

def rsynth(Bdata):
    #Train
    train=Bdata[1].loc[:Bdata[0][1],:]
    r1T_train=train.iloc[:,0]
    rmk_train=train.iloc[:,1]
    rC_train=train.iloc[:,2:]
    #Train
    test=Bdata[1].loc[Bdata[0][1]:Bdata[0][2],:]
    r1T_test=test.iloc[:,0]
    rmk_test=test.iloc[:,1]
    rC_test=test.iloc[:,2:]
    #Train
    event=Bdata[1].loc[Bdata[0][3]:,:]
    r1T_event=event.iloc[:,0]
    rmk_event=event.iloc[:,1]
    rC_event=event.iloc[:,2:]
    #2.ESTIMATORS
    #MARKET
    reg.fit(rmk_train.values.reshape(-1,1),r1T_train)
    #out-of-sample
    mkt_y_pred_os=reg.predict(rmk_test.values.reshape(-1,1)) #Potential outcome mkt model
    mkt_mse_test=mean_squared_error(r1T_test, mkt_y_pred_os)
    #event window
    mkt_r1NT=reg.predict(rmk_event.values.reshape(-1,1)) #Potential outcome mkt model
    #LASSO
    slreg.fit(rC_train,r1T_train)
    #out-of-sample
    lsw=slreg.coef_
    lsw=lsw/lsw.sum() #only if weights are positive
    lXX=np.matrix(rC_test)
    lsyn_y_pred_os=np.dot(lXX,lsw)
    lsyn_mse_test=mean_squared_error(r1T_test, lsyn_y_pred_os.transpose())
    #event window
    lXX_e=np.matrix(rC_event)
    lsyn_r1NT=np.dot(lXX_e,lsw)
    #ELASTIC NET
    enreg.fit(rC_train,r1T_train)
    #out-of-sample
    enw=enreg.coef_
    enw=enw/enw.sum() #only if weights are positive
    esyn_y_pred_os=np.dot(lXX,enw)
    esyn_mse_test=mean_squared_error(r1T_test, esyn_y_pred_os.transpose())
    #event window
    esyn_r1NT=np.dot(lXX_e,enw)
    #NAIVE
    #out-of-sample
    nsyn_y_pred_os=lXX.mean(1)
    nsyn_mse_test=mean_squared_error(r1T_test, nsyn_y_pred_os)
    #event window
    nsyn_r1NT=lXX_e.mean(1)
    #3.RESULTS
    #Treated return firm 1 and potential outcomes (non-treated)
    eventM=pd.DataFrame(np.matrix((r1T_event.values.tolist(),mkt_r1NT.tolist(),lsyn_r1NT.tolist()[0],
                   esyn_r1NT.tolist()[0],nsyn_r1NT.transpose().tolist()[0])).reshape(5,len(r1T_event)).transpose(),
                   index=r1T_event.index) #careful with 21, change
    eventM.columns=["RT","RNT_Market","RNT_Syn.Lasso","RNT_Syn.ENet","RNT_Syn.naive"]
    #treatment effects=abnormal returns
    AR=pd.DataFrame(np.repeat(eventM.values[:,0],4).reshape((len(eventM),4))-eventM.values[:,1:],index=r1T_event.index)
    AR.columns=["AR_Market","AR_Syn.Lasso","AR_Syn.ENet","AR_Syn.naive"]
    #individual test parametric(t.test) and non-parametric(wilcoxon)
    mkt_ttest=stats.ttest_ind(eventM.values[:,0],eventM.values[:,1], equal_var = False)
    mkt_wtest=stats.wilcoxon(eventM.values[:,0],eventM.values[:,1], zero_method='wilcox')
    lsyn_ttest=stats.ttest_ind(eventM.values[:,0],eventM.values[:,2], equal_var = False)
    lsyn_wtest=stats.wilcoxon(eventM.values[:,0],eventM.values[:,2], zero_method='wilcox')
    esyn_ttest=stats.ttest_ind(eventM.values[:,0],eventM.values[:,3], equal_var = False)
    esyn_wtest=stats.wilcoxon(eventM.values[:,0],eventM.values[:,3], zero_method='wilcox')
    nsyn_ttest=stats.ttest_ind(eventM.values[:,0],eventM.values[:,4], equal_var = False)
    nsyn_wtest=stats.wilcoxon(eventM.values[:,0],eventM.values[:,4], zero_method='wilcox')
    ttestp=[mkt_ttest.pvalue,lsyn_ttest.pvalue,esyn_ttest.pvalue,nsyn_ttest.pvalue]
    wtestp=[mkt_wtest.pvalue,lsyn_wtest.pvalue,esyn_wtest.pvalue,nsyn_wtest.pvalue]
    #Mean square error table testing window
    mseM=[mkt_mse_test,lsyn_mse_test,esyn_mse_test,nsyn_mse_test]
    model=["Market","Syn.Lasso","Syn.ENet","Syn.naive"]
    testM=pd.DataFrame(np.matrix((mseM,AR.sum(0).tolist(),ttestp,wtestp)).reshape(4,4).transpose(),index=model)
    testM.columns=['MSE_test.w','CAR','CAR_t.test','CAR_wilcoxon']
    out=(testM,eventM,AR)
    return out

In [5]:
#estimators sklearn
reg = linear_model.LinearRegression() #OLS
slreg = linear_model.Lasso(alpha=0.0001,fit_intercept=False, max_iter=1000,positive=True) # LASSO
enreg= linear_model.ElasticNet(alpha=0.0001, l1_ratio=0.5, fit_intercept=False,positive=True) #Enet

## 1. Read Data

In [6]:
#load global data
#event list has the pair date and firm id (adquierer and target)
filepath="acqux.csv"
Emadte=pd.read_csv(filepath,sep=",",parse_dates=[0],index_col="date")
#asset return time series (CRISP history)
with open('CRSP0016s.pickle', 'rb') as file:
    st16 = pickle.load(file)

In [7]:
st16.replace('NaN', np.NaN, inplace=True)

In [8]:
#inspect data event-list
#Emadte.info()
#Shorter sample  
EmadteS=Emadte.loc[slice('2006','2011'),:]
EmadteS.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5201 entries, 2006-01-02 to 2011-12-30
Data columns (total 1 columns):
permno    5201 non-null int64
dtypes: int64(1)
memory usage: 81.3 KB


In [9]:
#inspect data asset returns
#st16.info()
#Shorter sample 
st16=st16.loc[(slice(None),slice('2004','2012')),:]
st16.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15506755 entries, (10001, 2004-01-02 00:00:00) to (93436, 2012-12-31 00:00:00)
Data columns (total 2 columns):
RETX      float64
sprtrn    float64
dtypes: float64(2)
memory usage: 295.9 MB


## 2. Eliminate Missing Variables from universe of returns

In [10]:
#extract index
spmktix=st16.loc[(10001,slice('2004','2012')),'sprtrn']
spmktix=spmktix.values

In [11]:
#drop missing get balanced panel
st16.drop('sprtrn',axis=1)
st16.reset_index(inplace=True)
#pivot the table
st16=st16.pivot(index='date',columns='PERMNO',values='RETX')
st16.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2265 entries, 2004-01-02 to 2012-12-31
Columns: 11055 entries, 10001 to 93436
dtypes: float64(11055)
memory usage: 191.1 MB


In [12]:
st16.dropna(axis=1, how='any',inplace=True)
st16.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2265 entries, 2004-01-02 to 2012-12-31
Columns: 3352 entries, 10001 to 92874
dtypes: float64(3352)
memory usage: 57.9 MB


In [13]:
spmktix=pd.Series(spmktix,st16.index)
spmktix.name='sprtrn'
spmktix.head()

date
2004-01-02   -0.003094
2004-01-05    0.012395
2004-01-06    0.001292
2004-01-07    0.002367
2004-01-08    0.004963
Name: sprtrn, dtype: float64

In [14]:
#get the return data without missing variables
st16f=st16.stack()
len(st16f)

7592280

## 2. select event pairs (date and asset) with information in the return dataset

In [15]:
#Check that the dates in event list are in the return information
#full index of event dataset
eidxf=EmadteS.index
#unique
eidx=eidxf.unique()
#unique index of returndata set
ridx=st16f.index.get_level_values(level='date').unique()
#keep only dates that are in both event and return dataset
okidx=eidx[eidx.isin(ridx)]
#get the event list with dates that exist in the return dateset
EmadteOK=EmadteS[eidxf.isin(okidx)]
EmadteOK.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5158 entries, 2006-01-03 to 2011-12-30
Data columns (total 1 columns):
permno    5158 non-null int64
dtypes: int64(1)
memory usage: 80.6 KB


In [16]:
#Check that the assets in event list are in the return information
EmadteOK=EmadteOK[EmadteOK.permno.isin(st16f.index.get_level_values(level='PERMNO').unique())]
EmadteOK.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2967 entries, 2006-01-04 to 2011-12-30
Data columns (total 1 columns):
permno    2967 non-null int64
dtypes: int64(1)
memory usage: 46.4 KB


In [17]:
EmadteOK.head()

,permno
date,
2006-01-04,86013
2006-01-04,86717
2006-01-04,21742
2006-01-05,77178
2006-01-05,25778


In [18]:
EmadteOKS=pd.Series(EmadteOK.permno.values,index=EmadteOK.index)
EmadteOKS.head()

date
2006-01-04    86013
2006-01-04    86717
2006-01-04    21742
2006-01-05    77178
2006-01-05    25778
dtype: int64

### Merger Announcements
On February 1, 2008, Microsoft unveiled a hostile offer for Yahoo Inc (YHOO) of $44.6$ billion. Microsoft Corp’s (MSFT) offer of $31$ dollars per Yahoo share consisted of one-half cash and one-half Microsoft shares, and represented a $62\%$ premium to Yahoo’s closing price on the previous day. However, Yahoo’s board of directors – led by co-founder Jerry Yang – rejected Microsoft’s offer, saying that it substantially undervalued the company. Unfortunately, the credit crisis that gripped the world later that year also took its toll on Yahoo shares, resulting in the stock trading below $10$ by November 2008. Yahoo’s subsequent road to recovery was a long one, and the stock only exceeded Microsoft’s original $31$ offer five and a half years later in September 2013.

Read more: How Mergers and Acquisitions Can Affect A Company | Investopedia http://www.investopedia.com/articles/investing/102914/how-mergers-and-acquisitions-can-affect-company.asp#ixzz4sZ1VgjjF 

In [21]:
#Find Microsoft, pemno is 10107
EmadteOKS['2008-02-01']

date
2008-02-01    10107
2008-02-01    79103
2008-02-01    63335
2008-02-01    85176
2008-02-01    81740
dtype: int64

In [38]:
pairs=EmadteOK
pairs.reset_index(inplace=True)
pairs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2967 entries, 0 to 2966
Data columns (total 2 columns):
date      2967 non-null datetime64[ns]
permno    2967 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 46.4 KB


In [39]:
#query id
MSeventid=EmadteOKS.index.get_loc('2008-02-01').start
pair1=pairs.iloc[MSeventid,:]
pair1

date      2008-02-01 00:00:00
permno                  10107
Name: 1357, dtype: object

## 1.Build Data

In [40]:
Bdata1=buildD(pair1)

In [41]:
#event timeline
Bdata1[0]

(Timestamp('2007-02-27 00:00:00'),
 Timestamp('2007-10-01 00:00:00'),
 Timestamp('2007-12-11 00:00:00'),
 Timestamp('2008-01-17 00:00:00'),
 Timestamp('2008-02-01 00:00:00'),
 Timestamp('2008-02-15 00:00:00'))

In [42]:
#Data
Bdata1[1].head()

,10107,sprtrn,10001,10025,10032,10044,10051,10065,10100,10138,...,90916,90983,91287,91556,91626,91855,92583,92690,92807,92874
date,,,,,,,,,,,,,,,,,,,,,
2007-02-27,-0.041280,-0.034725,0.003571,-0.038251,-0.038128,-0.035600,-0.046414,-0.025514,-0.034866,-0.046676,...,-0.010475,-0.005455,-0.113435,-0.028622,-0.032856,-0.043118,-0.144250,-0.005567,-0.002146,-0.017544
2007-02-28,0.010764,0.005561,0.009964,0.030000,-0.015015,0.006151,0.032743,0.001447,-0.074346,0.017705,...,-0.007329,-0.010969,0.025638,-0.004556,0.020518,0.002163,0.047794,-0.005731,-0.004301,-0.016484
2007-03-01,-0.002840,-0.002595,0.000000,-0.026037,0.009756,-0.004396,-0.034276,-0.000719,0.023756,-0.004940,...,-0.003281,-0.007394,-0.026248,-0.004272,0.008570,0.007914,-0.056140,-0.006702,0.010799,0.017360
2007-03-02,-0.011748,-0.011403,0.000705,-0.026280,-0.020531,-0.006623,-0.006211,-0.013074,0.000000,-0.010361,...,-0.009877,-0.001122,-0.026190,-0.009501,-0.007190,-0.062812,-0.022305,-0.004723,-0.012821,-0.001963
2007-03-05,-0.007568,-0.009408,-0.027465,0.000931,-0.018496,-0.003704,-0.025893,-0.008844,-0.013812,-0.027350,...,-0.027431,-0.007491,-0.074609,-0.017327,-0.056945,-0.019802,-0.012928,-0.014237,0.004329,-0.011142


In [44]:
#save data for Example
Bdata1[1].to_pickle('E10107.pickle')

In [45]:
#Make Event dates panda series
stage=["T1","T2","T3","T4",'Event_0','T5']
#dated=[dinvl[0],dinvl[1],dinvl[2],dinvl[len(dinvl)-1]]
evDtes=dict(zip(stage,Bdata1[0]))
evDtes=pd.Series(evDtes, index=stage)
evDtes.name='date'
evDtes

T1        2007-02-27
T2        2007-10-01
T3        2007-12-11
T4        2008-01-17
Event_0   2008-02-01
T5        2008-02-15
Name: date, dtype: datetime64[ns]

In [46]:
evDtes.to_csv('DatesE10107.csv')

## 2. Estimation 

In [ ]:
Sout1=rsynth(Bdata1)

In [ ]:
Sout1[0]

In [ ]:
Sout1[1]

In [ ]:
Sout1[2]